# RBPF Comprehensive Comparison Analysis

Compares 4 configurations:
1. **Baseline** - RBPF alone, no parameter learning
2. **Liu-West** - RBPF + Liu-West shrinkage learning
3. **Storvik** - RBPF + Sleeping Storvik learning
4. **Storvik+APF** - Full stack with lookahead

Analysis covers:
- Volatility tracking accuracy
- Regime detection performance
- Parameter learning convergence
- Latency distribution
- Scenario-specific behavior

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.gridspec import GridSpec

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 10

# Color scheme for 4 modes
COLORS = {
    'baseline': '#1f77b4',    # Blue
    'liu_west': '#ff7f0e',    # Orange
    'storvik': '#2ca02c',     # Green
    'storvik_apf': '#d62728'  # Red
}
LABELS = {
    'baseline': 'Baseline',
    'liu_west': 'Liu-West',
    'storvik': 'Storvik',
    'storvik_apf': 'Storvik+APF'
}

# Load data
dfs = {
    'baseline': pd.read_csv('rbpf_baseline.csv'),
    'liu_west': pd.read_csv('rbpf_liu_west.csv'),
    'storvik': pd.read_csv('rbpf_storvik.csv'),
    'storvik_apf': pd.read_csv('rbpf_storvik_apf.csv')
}
df_summary = pd.read_csv('rbpf_comparison_summary.csv')

print(f"Loaded {len(dfs['baseline'])} ticks per mode")
print(f"\nSummary metrics:")
display(df_summary)

## 1. Volatility Tracking Comparison

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(16, 12), sharex=True)

# Use baseline for ground truth
df_base = dfs['baseline']

# True volatility
ax = axes[0]
ax.plot(df_base['tick'], df_base['true_vol'], 'k-', linewidth=1, label='True Vol', alpha=0.8)
for key in dfs:
    ax.plot(dfs[key]['tick'], dfs[key]['est_vol'], color=COLORS[key], 
            linewidth=0.7, alpha=0.6, label=LABELS[key])
ax.set_ylabel('Volatility')
ax.set_title('Volatility Tracking: All Methods')
ax.legend(loc='upper right', ncol=5)
ax.set_ylim(0, 0.5)

# Log-volatility
ax = axes[1]
ax.plot(df_base['tick'], df_base['true_log_vol'], 'k-', linewidth=1, label='True', alpha=0.8)
for key in dfs:
    ax.plot(dfs[key]['tick'], dfs[key]['est_log_vol'], color=COLORS[key], 
            linewidth=0.7, alpha=0.6, label=LABELS[key])
ax.set_ylabel('Log-Volatility')
ax.set_title('Log-Volatility Tracking')
ax.legend(loc='upper right', ncol=5)

# True regime (background)
ax = axes[2]
ax.fill_between(df_base['tick'], df_base['true_regime'], alpha=0.3, color='gray', label='True Regime')
for key in dfs:
    ax.plot(dfs[key]['tick'], dfs[key]['est_regime'], color=COLORS[key], 
            linewidth=0.7, alpha=0.7, label=LABELS[key])
ax.set_ylabel('Regime')
ax.set_xlabel('Tick')
ax.set_title('Regime Detection')
ax.set_yticks([0, 1, 2, 3])
ax.legend(loc='upper right', ncol=5)

# Add scenario markers
scenario_boundaries = [1000, 2000, 2500, 3500, 4500, 5000]
for ax in axes:
    for b in scenario_boundaries:
        ax.axvline(b, color='gray', linestyle='--', alpha=0.3)

plt.tight_layout()
plt.savefig('rbpf_comparison_overview.png', dpi=150)
plt.show()

## 2. Tracking Error Analysis

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Compute errors
errors = {}
for key in dfs:
    errors[key] = dfs[key]['est_log_vol'] - dfs[key]['true_log_vol']

# Rolling RMSE (window=100)
ax = axes[0, 0]
window = 100
for key in dfs:
    rolling_rmse = np.sqrt((errors[key]**2).rolling(window).mean())
    ax.plot(dfs[key]['tick'], rolling_rmse, color=COLORS[key], label=LABELS[key], alpha=0.8)
ax.set_ylabel('Rolling RMSE')
ax.set_xlabel('Tick')
ax.set_title(f'Log-Vol Rolling RMSE (window={window})')
ax.legend()

# Error distribution (histogram)
ax = axes[0, 1]
for key in dfs:
    ax.hist(errors[key], bins=50, alpha=0.5, color=COLORS[key], label=LABELS[key])
ax.set_xlabel('Log-Vol Error')
ax.set_ylabel('Count')
ax.set_title('Error Distribution')
ax.legend()

# RMSE bar chart
ax = axes[1, 0]
rmse_values = [np.sqrt((errors[key]**2).mean()) for key in dfs]
bars = ax.bar(range(4), rmse_values, color=[COLORS[k] for k in dfs])
ax.set_xticks(range(4))
ax.set_xticklabels([LABELS[k] for k in dfs])
ax.set_ylabel('RMSE')
ax.set_title('Log-Vol RMSE Comparison')
for i, v in enumerate(rmse_values):
    ax.text(i, v + 0.005, f'{v:.4f}', ha='center', fontsize=9)

# MAE bar chart
ax = axes[1, 1]
mae_values = [errors[key].abs().mean() for key in dfs]
bars = ax.bar(range(4), mae_values, color=[COLORS[k] for k in dfs])
ax.set_xticks(range(4))
ax.set_xticklabels([LABELS[k] for k in dfs])
ax.set_ylabel('MAE')
ax.set_title('Log-Vol MAE Comparison')
for i, v in enumerate(mae_values):
    ax.text(i, v + 0.005, f'{v:.4f}', ha='center', fontsize=9)

plt.tight_layout()
plt.savefig('rbpf_comparison_errors.png', dpi=150)
plt.show()

# Print statistics
print("\nLog-Vol Error Statistics:")
print("=" * 60)
for key in dfs:
    e = errors[key]
    print(f"\n{LABELS[key]:15s}: RMSE={np.sqrt((e**2).mean()):.4f}, MAE={e.abs().mean():.4f}, Bias={e.mean():.4f}")

## 3. Regime Detection Performance

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Regime accuracy over time (rolling)
ax = axes[0, 0]
window = 200
for key in dfs:
    correct = (dfs[key]['est_regime'] == dfs[key]['true_regime']).astype(float)
    rolling_acc = correct.rolling(window).mean() * 100
    ax.plot(dfs[key]['tick'], rolling_acc, color=COLORS[key], label=LABELS[key], alpha=0.8)
ax.set_ylabel('Accuracy (%)')
ax.set_xlabel('Tick')
ax.set_title(f'Rolling Regime Accuracy (window={window})')
ax.legend()
ax.set_ylim(0, 100)

# Overall accuracy bar chart
ax = axes[0, 1]
acc_values = [(dfs[key]['est_regime'] == dfs[key]['true_regime']).mean() * 100 for key in dfs]
bars = ax.bar(range(4), acc_values, color=[COLORS[k] for k in dfs])
ax.set_xticks(range(4))
ax.set_xticklabels([LABELS[k] for k in dfs])
ax.set_ylabel('Accuracy (%)')
ax.set_title('Overall Regime Accuracy')
for i, v in enumerate(acc_values):
    ax.text(i, v + 1, f'{v:.1f}%', ha='center', fontsize=9)

# Confusion matrix for best method (Storvik)
from sklearn.metrics import confusion_matrix
ax = axes[1, 0]
df_best = dfs['storvik']
cm = confusion_matrix(df_best['true_regime'], df_best['est_regime'], labels=[0,1,2,3])
cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
sns.heatmap(cm_norm, annot=True, fmt='.2f', cmap='Greens', ax=ax,
            xticklabels=['R0', 'R1', 'R2', 'R3'],
            yticklabels=['R0', 'R1', 'R2', 'R3'])
ax.set_xlabel('Predicted')
ax.set_ylabel('True')
ax.set_title('Confusion Matrix (Storvik)')

# ESS comparison
ax = axes[1, 1]
ess_values = [dfs[key]['ess'].mean() for key in dfs]
bars = ax.bar(range(4), ess_values, color=[COLORS[k] for k in dfs])
ax.set_xticks(range(4))
ax.set_xticklabels([LABELS[k] for k in dfs])
ax.set_ylabel('Average ESS')
ax.set_title('Effective Sample Size')
for i, v in enumerate(ess_values):
    ax.text(i, v + 5, f'{v:.0f}', ha='center', fontsize=9)

plt.tight_layout()
plt.savefig('rbpf_comparison_regime.png', dpi=150)
plt.show()

## 4. Parameter Learning Convergence

In [ ]:
# Ground truth parameters
TRUE_MU_VOL = {0: -4.6, 1: -3.5, 2: -2.5, 3: -1.5}
TRUE_SIGMA_VOL = {0: 0.05, 1: 0.10, 2: 0.20, 3: 0.35}

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# mu_vol R0 learning
ax = axes[0, 0]
ax.axhline(TRUE_MU_VOL[0], color='black', linestyle='--', linewidth=2, label='True μ_vol R0')
for key in ['liu_west', 'storvik', 'storvik_apf']:
    if 'learned_mu_vol_r0' in dfs[key].columns:
        ax.plot(dfs[key]['tick'], dfs[key]['learned_mu_vol_r0'], 
                color=COLORS[key], label=LABELS[key], alpha=0.8)
ax.set_ylabel('μ_vol (R0)')
ax.set_xlabel('Tick')
ax.set_title('Parameter Learning: μ_vol (Calm Regime)')
ax.legend()

# mu_vol R3 learning
ax = axes[0, 1]
ax.axhline(TRUE_MU_VOL[3], color='black', linestyle='--', linewidth=2, label='True μ_vol R3')
for key in ['liu_west', 'storvik', 'storvik_apf']:
    if 'learned_mu_vol_r3' in dfs[key].columns:
        ax.plot(dfs[key]['tick'], dfs[key]['learned_mu_vol_r3'], 
                color=COLORS[key], label=LABELS[key], alpha=0.8)
ax.set_ylabel('μ_vol (R3)')
ax.set_xlabel('Tick')
ax.set_title('Parameter Learning: μ_vol (Crisis Regime)')
ax.legend()

# Final parameter error comparison
ax = axes[1, 0]
keys_learn = ['liu_west', 'storvik', 'storvik_apf']
mu_r0_errors = []
mu_r3_errors = []
for key in keys_learn:
    if 'learned_mu_vol_r0' in dfs[key].columns:
        final_mu_r0 = dfs[key]['learned_mu_vol_r0'].iloc[-1]
        final_mu_r3 = dfs[key]['learned_mu_vol_r3'].iloc[-1]
        mu_r0_errors.append(abs(final_mu_r0 - TRUE_MU_VOL[0]))
        mu_r3_errors.append(abs(final_mu_r3 - TRUE_MU_VOL[3]))
    else:
        mu_r0_errors.append(0)
        mu_r3_errors.append(0)

x = np.arange(len(keys_learn))
width = 0.35
ax.bar(x - width/2, mu_r0_errors, width, label='R0 Error', color='steelblue')
ax.bar(x + width/2, mu_r3_errors, width, label='R3 Error', color='coral')
ax.set_xticks(x)
ax.set_xticklabels([LABELS[k] for k in keys_learn])
ax.set_ylabel('|Error|')
ax.set_title('Final μ_vol Parameter Error')
ax.legend()

# Learning speed (ticks to reach within 10% of true)
ax = axes[1, 1]
# Placeholder - would need to compute convergence time
ax.text(0.5, 0.5, 'Learning\nConvergence\nAnalysis', 
        ha='center', va='center', fontsize=14, transform=ax.transAxes)
ax.set_title('Convergence Speed (placeholder)')

plt.tight_layout()
plt.savefig('rbpf_comparison_learning.png', dpi=150)
plt.show()

# Print final learned parameters
print("\nFinal Learned Parameters:")
print("=" * 60)
print(f"{'Method':15s} {'μ_vol R0':>12s} {'True':>8s} {'μ_vol R3':>12s} {'True':>8s}")
print("-" * 60)
for key in keys_learn:
    if 'learned_mu_vol_r0' in dfs[key].columns:
        mu_r0 = dfs[key]['learned_mu_vol_r0'].iloc[-1]
        mu_r3 = dfs[key]['learned_mu_vol_r3'].iloc[-1]
        print(f"{LABELS[key]:15s} {mu_r0:12.4f} {TRUE_MU_VOL[0]:8.2f} {mu_r3:12.4f} {TRUE_MU_VOL[3]:8.2f}")

## 5. Latency Analysis

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Histogram comparison
ax = axes[0, 0]
for key in dfs:
    ax.hist(dfs[key]['latency_us'], bins=50, alpha=0.5, color=COLORS[key], 
            label=f"{LABELS[key]} (μ={dfs[key]['latency_us'].mean():.1f}μs)")
ax.set_xlabel('Latency (μs)')
ax.set_ylabel('Count')
ax.set_title('Latency Distribution')
ax.legend()

# Box plot
ax = axes[0, 1]
data_box = [dfs[key]['latency_us'] for key in dfs]
bp = ax.boxplot(data_box, labels=[LABELS[k] for k in dfs], patch_artist=True)
for i, (box, key) in enumerate(zip(bp['boxes'], dfs.keys())):
    box.set_facecolor(COLORS[key])
    box.set_alpha(0.6)
ax.set_ylabel('Latency (μs)')
ax.set_title('Latency Box Plot')

# Percentile comparison
ax = axes[1, 0]
percentiles = [50, 75, 90, 95, 99]
x = np.arange(len(percentiles))
width = 0.2
for i, key in enumerate(dfs):
    p_vals = [np.percentile(dfs[key]['latency_us'], p) for p in percentiles]
    ax.bar(x + i*width - 1.5*width, p_vals, width, label=LABELS[key], color=COLORS[key], alpha=0.8)
ax.set_xticks(x)
ax.set_xticklabels([f'P{p}' for p in percentiles])
ax.set_ylabel('Latency (μs)')
ax.set_title('Latency Percentiles')
ax.legend()

# Average latency bar
ax = axes[1, 1]
avg_lat = [dfs[key]['latency_us'].mean() for key in dfs]
p99_lat = [dfs[key]['latency_us'].quantile(0.99) for key in dfs]
x = np.arange(4)
width = 0.35
ax.bar(x - width/2, avg_lat, width, label='Mean', color='steelblue')
ax.bar(x + width/2, p99_lat, width, label='P99', color='coral')
ax.set_xticks(x)
ax.set_xticklabels([LABELS[k] for k in dfs])
ax.set_ylabel('Latency (μs)')
ax.set_title('Mean vs P99 Latency')
ax.legend()

plt.tight_layout()
plt.savefig('rbpf_comparison_latency.png', dpi=150)
plt.show()

# Print latency statistics
print("\nLatency Statistics:")
print("=" * 70)
print(f"{'Method':15s} {'Mean':>10s} {'Median':>10s} {'P95':>10s} {'P99':>10s} {'Max':>10s}")
print("-" * 70)
for key in dfs:
    lat = dfs[key]['latency_us']
    print(f"{LABELS[key]:15s} {lat.mean():10.2f} {lat.median():10.2f} {lat.quantile(0.95):10.2f} {lat.quantile(0.99):10.2f} {lat.max():10.2f}")

## 6. Scenario-Specific Analysis

In [ ]:
# Define scenarios
scenarios = [
    ('1. Calm', 0, 1000),
    ('2. Gradual Rise', 1000, 2000),
    ('3. Sudden Crisis', 2000, 2500),
    ('4. Crisis Persist', 2500, 3500),
    ('5. Recovery', 3500, 4500),
    ('6. Flash Crash', 4500, 5000),
    ('7. Choppy', 5000, 7000)
]

# Compute metrics per scenario
results = []
for name, start, end in scenarios:
    row = {'Scenario': name}
    for key in dfs:
        mask = (dfs[key]['tick'] >= start) & (dfs[key]['tick'] < end)
        df_slice = dfs[key][mask]
        
        # Regime accuracy
        acc = (df_slice['est_regime'] == df_slice['true_regime']).mean() * 100
        row[f'{LABELS[key]} Acc'] = f'{acc:.1f}%'
        
        # Log-vol RMSE
        rmse = np.sqrt(((df_slice['est_log_vol'] - df_slice['true_log_vol'])**2).mean())
        row[f'{LABELS[key]} RMSE'] = f'{rmse:.3f}'
    
    results.append(row)

df_scenarios = pd.DataFrame(results)
print("\nScenario-Specific Performance:")
print("=" * 100)
display(df_scenarios)

In [ ]:
# Scenario accuracy heatmap
fig, ax = plt.subplots(figsize=(10, 6))

# Build accuracy matrix
acc_matrix = []
for name, start, end in scenarios:
    row = []
    for key in dfs:
        mask = (dfs[key]['tick'] >= start) & (dfs[key]['tick'] < end)
        acc = (dfs[key][mask]['est_regime'] == dfs[key][mask]['true_regime']).mean() * 100
        row.append(acc)
    acc_matrix.append(row)

acc_matrix = np.array(acc_matrix)
sns.heatmap(acc_matrix, annot=True, fmt='.1f', cmap='RdYlGn',
            xticklabels=[LABELS[k] for k in dfs],
            yticklabels=[s[0] for s in scenarios],
            vmin=40, vmax=100, ax=ax)
ax.set_title('Regime Accuracy by Scenario (%)')
ax.set_xlabel('Method')
ax.set_ylabel('Scenario')

plt.tight_layout()
plt.savefig('rbpf_comparison_scenario_heatmap.png', dpi=150)
plt.show()

## 7. Crisis Zoom: Sudden Crisis (Scenario 3)

In [ ]:
# Zoom into sudden crisis transition
fig, axes = plt.subplots(3, 1, figsize=(14, 10), sharex=True)

mask_range = (1900, 2200)  # Around the R2→R3 transition

for key in dfs:
    mask = (dfs[key]['tick'] >= mask_range[0]) & (dfs[key]['tick'] < mask_range[1])
    df_zoom = dfs[key][mask]
    
    # Volatility
    axes[0].plot(df_zoom['tick'], df_zoom['est_vol'], color=COLORS[key], 
                 label=LABELS[key], alpha=0.8, linewidth=1.5)
    
    # Regime
    axes[1].plot(df_zoom['tick'], df_zoom['est_regime'], color=COLORS[key], 
                 label=LABELS[key], alpha=0.8, linewidth=1.5)
    
    # ESS
    axes[2].plot(df_zoom['tick'], df_zoom['ess'], color=COLORS[key], 
                 label=LABELS[key], alpha=0.8, linewidth=1.5)

# Add ground truth
df_base = dfs['baseline']
mask = (df_base['tick'] >= mask_range[0]) & (df_base['tick'] < mask_range[1])
axes[0].plot(df_base[mask]['tick'], df_base[mask]['true_vol'], 'k--', linewidth=2, label='True')
axes[1].fill_between(df_base[mask]['tick'], df_base[mask]['true_regime'], alpha=0.2, color='gray')

axes[0].set_ylabel('Volatility')
axes[0].set_title('Sudden Crisis Transition (tick 2000)')
axes[0].legend(loc='upper left', ncol=5)
axes[0].axvline(2000, color='red', linestyle=':', alpha=0.7)

axes[1].set_ylabel('Regime')
axes[1].set_yticks([0, 1, 2, 3])
axes[1].legend(loc='upper left', ncol=5)
axes[1].axvline(2000, color='red', linestyle=':', alpha=0.7)

axes[2].set_ylabel('ESS')
axes[2].set_xlabel('Tick')
axes[2].legend(loc='upper left', ncol=5)
axes[2].axvline(2000, color='red', linestyle=':', alpha=0.7)

plt.tight_layout()
plt.savefig('rbpf_comparison_crisis_zoom.png', dpi=150)
plt.show()

## 8. Flash Crash Zoom (Scenario 6)

In [ ]:
# Zoom into flash crash
fig, axes = plt.subplots(3, 1, figsize=(14, 10), sharex=True)

mask_range = (4650, 4850)  # Flash crash at 4700-4750

for key in dfs:
    mask = (dfs[key]['tick'] >= mask_range[0]) & (dfs[key]['tick'] < mask_range[1])
    df_zoom = dfs[key][mask]
    
    axes[0].plot(df_zoom['tick'], df_zoom['est_vol'], color=COLORS[key], 
                 label=LABELS[key], alpha=0.8, linewidth=1.5)
    axes[1].plot(df_zoom['tick'], df_zoom['est_regime'], color=COLORS[key], 
                 label=LABELS[key], alpha=0.8, linewidth=1.5)
    axes[2].plot(df_zoom['tick'], df_zoom['ess'], color=COLORS[key], 
                 label=LABELS[key], alpha=0.8, linewidth=1.5)

# Ground truth
df_base = dfs['baseline']
mask = (df_base['tick'] >= mask_range[0]) & (df_base['tick'] < mask_range[1])
axes[0].plot(df_base[mask]['tick'], df_base[mask]['true_vol'], 'k--', linewidth=2, label='True')
axes[1].fill_between(df_base[mask]['tick'], df_base[mask]['true_regime'], alpha=0.2, color='gray')

# Mark flash crash region
for ax in axes:
    ax.axvspan(4700, 4750, alpha=0.2, color='red')

axes[0].set_ylabel('Volatility')
axes[0].set_title('Flash Crash (ticks 4700-4750)')
axes[0].legend(loc='upper right', ncol=5)

axes[1].set_ylabel('Regime')
axes[1].set_yticks([0, 1, 2, 3])
axes[1].legend(loc='upper right', ncol=5)

axes[2].set_ylabel('ESS')
axes[2].set_xlabel('Tick')
axes[2].legend(loc='upper right', ncol=5)

plt.tight_layout()
plt.savefig('rbpf_comparison_flash_crash.png', dpi=150)
plt.show()

## 9. Summary Comparison Table

In [ ]:
# Build comprehensive summary
summary_data = {
    'Metric': [
        'Log-Vol RMSE',
        'Log-Vol MAE',
        'Vol RMSE',
        'Regime Accuracy',
        'Avg ESS',
        'Min ESS',
        'Avg Latency (μs)',
        'P99 Latency (μs)',
        'Max Latency (μs)'
    ]
}

for key in dfs:
    df = dfs[key]
    log_err = df['est_log_vol'] - df['true_log_vol']
    vol_err = df['est_vol'] - df['true_vol']
    
    summary_data[LABELS[key]] = [
        f"{np.sqrt((log_err**2).mean()):.4f}",
        f"{log_err.abs().mean():.4f}",
        f"{np.sqrt((vol_err**2).mean()):.6f}",
        f"{(df['est_regime'] == df['true_regime']).mean()*100:.1f}%",
        f"{df['ess'].mean():.1f}",
        f"{df['ess'].min():.1f}",
        f"{df['latency_us'].mean():.2f}",
        f"{df['latency_us'].quantile(0.99):.2f}",
        f"{df['latency_us'].max():.2f}"
    ]

df_final = pd.DataFrame(summary_data)
print("\n" + "="*80)
print("COMPREHENSIVE SUMMARY")
print("="*80)
display(df_final)

# Highlight winner for each metric
print("\n" + "="*80)
print("CONCLUSIONS")
print("="*80)
print("""
Key findings:
1. Parameter Learning: Compare Liu-West vs Storvik convergence and final error
2. Regime Detection: Which method tracks regime changes fastest?
3. Latency: Does Storvik add significant overhead vs Liu-West?
4. APF Benefit: Does lookahead help on top of Storvik?
""")

In [ ]:
# Save all results
print("\nSaved figures:")
print("  - rbpf_comparison_overview.png")
print("  - rbpf_comparison_errors.png")
print("  - rbpf_comparison_regime.png")
print("  - rbpf_comparison_learning.png")
print("  - rbpf_comparison_latency.png")
print("  - rbpf_comparison_scenario_heatmap.png")
print("  - rbpf_comparison_crisis_zoom.png")
print("  - rbpf_comparison_flash_crash.png")